# 04 · API Demo (FastAPI)

REST API pro doporučení her pro **event** (počet hráčů, délka, tagy).

• Endpoint: `GET /rec/event?players=4&duration=60&tags=cooperative&tags=party&k=5`  
• Data: `data/games_features.parquet` (z 01)  
• Skórování: 0.7 × podobnost TF-IDF tagů + 0.3 × popularita `pop_norm`

### Spuštění mimo Jupyter
1) Ulož druhou kódovou buňku (FastAPI app) do `app.py`  
2) V terminálu spusť:  
```bash
uvicorn app:app --reload
```
3) Otevři http://127.0.0.1:8000/docs a zkus `/rec/event`

In [ ]:
# (Volitelné) instalace balíčků. Na Windows je spolehlivé použít Anacondu:
# conda install -c conda-forge fastapi uvicorn scikit-learn pandas numpy pyarrow
try:
    from fastapi import FastAPI, Query
except Exception:
    raise RuntimeError("Chybí FastAPI. Nainstaluj prosím: pip install fastapi uvicorn  (nebo conda install -c conda-forge fastapi uvicorn)")

import os, numpy as np, pandas as pd
from typing import List, Optional
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

DATA_DIR = "data"
games_path = os.path.join(DATA_DIR, "games_features.parquet")
assert os.path.exists(games_path), "Chybí data/games_features.parquet – spusť 01_feature_engineering.ipynb"

# Načteme hry a připravíme TF-IDF nad tagy (kategorie + mechaniky)
games = pd.read_parquet(games_path)
games["tags"] = (games["categories"].fillna("") + " " + games["mechanics"].fillna("")).str.lower()
vec = TfidfVectorizer(token_pattern=r"[a-zA-Z\-]+")
X = vec.fit_transform(games["tags"])  # řidká matice TF-IDF

def candidates(players:int, duration:int, tags:Optional[List[str]], topk:int=50, time_tolerance:float=1.2):
    g = games.copy()
    ok = (g["min_players"] <= players) & (g["max_players"] >= players) & (g["playing_time"] <= int(duration * time_tolerance))
    g = g[ok].copy()
    if tags:
        q = " ".join(t.lower() for t in tags)
        qv = vec.transform([q])
        g["sim"] = cosine_similarity(X[g.index], qv).ravel()
    else:
        g["sim"] = 0.0
    g["cand_score"] = 0.7 * g["sim"] + 0.3 * g.get("pop_norm", 0).fillna(0)
    return g.sort_values("cand_score", ascending=False).head(topk)

# --- FastAPI aplikace ---
app = FastAPI(title="Boardgame Event Recommender API")

@app.get("/rec/event")
def rec_event(players:int, duration:int, tags: Optional[List[str]] = Query(default=None), k:int=5):
    df = candidates(players, duration, tags, topk=50)
    df = df.copy()
    df["score"] = df["cand_score"]
    cols = ["name","min_players","max_players","playing_time","weight","categories","mechanics","score"]
    return df.sort_values("score", ascending=False).head(k)[cols].to_dict(orient="records")

print("API připraveno. Uložte tuto buňku jako app.py a spusťte: uvicorn app:app --reload")

## Rychlé lokální otestování bez uvicornu (volitelné)
Níže si můžeš zavolat stejné funkce přímo v notebooku, abys viděl, co API vrátí.

In [ ]:
def preview(players, duration, tags, k=5):
    from pprint import pprint
    out = candidates(players, duration, tags, topk=50)
    out = out.copy()
    out["score"] = 0.7*out["sim"] + 0.3*out.get("pop_norm",0).fillna(0)
    cols = ["name","min_players","max_players","playing_time","weight","categories","mechanics","score"]
      
    return out.sort_values("score", ascending=False).head(k)[cols]

preview(4, 60, ["cooperative"], k=5)